In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import rasterio
from rasterio.plot import show
from geodatasets import get_path
import numpy as np
from matplotlib.colors import ListedColormap

import rioxarray
import xarray
from shapely.geometry import box
from geocube.vector import vectorize

In [3]:
df = pd.read_excel('./GIS_data_code.xlsx', sheet_name=1, header=0)
df = df.rename(columns={'Longitude (degrees)' : 'Longitude',
                   'Latitude (degrees)' : 'Latitude'})
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude),
    crs='EPSG:4326' #GPS가 사용하는 좌표계
)

In [4]:
tif_path = "./ucl_2014_v2-3.tif"

'''with rasterio.open(tif_path) as src:
    metadata = src.meta

    raster_data= src.read(1)
'''

df_crop = rioxarray.open_rasterio(tif_path).squeeze()
df_crop.name = "crop_index"
#gdf_crop = vectorize(df_crop)
df_crop

<xarray.DataArray 'crop_index' (y: 20823, x: 43200)>
[899553600 values with dtype=uint8]
Coordinates:
    band         int64 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... -90.01 -90.0 -90.0
  * y            (y) float64 83.63 83.62 83.62 83.62 ... 40.25 40.25 40.25 40.25
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     255
    scale_factor:   1.0
    add_offset:     0.0

In [5]:
gdf_crop = vectorize(df_crop)

In [6]:
gdf_cropland = gdf_crop.groupby('crop_index')['geometry'].apply(lambda x: x.unary_union)

In [ ]:
from pyproj import CRS

# Assuming 'gdf' is your GeoDataFrame
# Set the CRS - For example, EPSG:4326 (WGS84)
gdf_cropland.crs = CRS.from_epsg(4326)

In [ ]:
gdf_cropland.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
# gdf_pasture.to_crs('EPSG:4326')

In [ ]:
buffer_radius = 10
buffer_gdf = gdf.copy()
buffer_gdf =buffer_gdf.to_crs('EPSG:3857')
buffer_gdf['geometry'] = buffer_gdf.buffer(buffer_radius*1000)
buffer_gdf['geometry_20'] = buffer_gdf.buffer(20*1000)
buffer_gdf['geometry_100'] = buffer_gdf.buffer(100*1000)

In [ ]:
buffer_gdf

,Unnamed: 0.1,Unnamed: 0,Property Name,Property ID,State/Province,Country Name,SNL Global Region,Latitude,Longitude,Population density in a 10km radius -Mean,...,Arid climate,Tropical rainforest climate,Polar climate,Croplands in 10km radius - Mean,Croplands in 20km radius - Mean,Croplands in 100km radius - Mean,Pasture land in a 10 km radius - Mean,Pasture land in a 20 km radius - Mean,Pasture land in a 100 km radius - Mean,geometry
0,0,0,101,64665,Santa Catarina,Brazil,Latin America and Caribbean,-28.732400,-49.245520,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.151388,0.138442,NaN,"POLYGON ((-5381986.210 -3341630.440, -5382467...."
1,1,1,11 RAU-Novaya,68289,Luhans'ka Oblast',Ukraine,Europe,48.519540,38.658270,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.069485,0.115809,NaN,"POLYGON ((4403418.931 6193727.142, 4402937.404..."
2,2,2,1165L,63154,Tete,Mozambique,Africa,-16.230000,34.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.077611,0.247622,NaN,"POLYGON ((3884862.687 -1831373.468, 3884381.16..."
3,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.482790,-58.833530,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.443950,0.375373,NaN,"POLYGON ((-6449318.601 -1286902.183, -6449800...."
4,4,4,12 Mile Creek,66178,Queensland,Australia,Asia-Pacific,-23.163000,149.083000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.996582,0.955124,NaN,"POLYGON ((16695843.646 -2651742.668, 16695362...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34264,34264,34264,Zymo,38983,British Columbia,Canada,United States and Canada,54.831119,-128.058003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-14155351.686 7329158.500, -14155833..."
34265,34265,34265,Zyrianovsk Complex,28934,Shyghys Qazaqstan oblysy,Kazakhstan,Europe,49.877040,84.262650,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.521052,0.513982,NaN,"POLYGON ((9480075.291 6425008.504, 9479593.764..."
34266,34266,34266,Zyrox Texada,59481,British Columbia,Canada,United States and Canada,49.775650,-124.600290,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.001141,0.002642,NaN,"POLYGON ((-13770440.835 6407512.648, -13770922..."
34267,34267,34267,Zyryanovskoe,61205,Shyghys Qazaqstan oblysy,Kazakhstan,Europe,49.740000,84.296000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.706592,0.699720,NaN,"POLYGON ((9483787.796 6401369.585, 9483306.269..."


In [ ]:
gdf[gdf['Latitude'].isnull()]['geometry']

1292     POINT EMPTY
3827     POINT EMPTY
4018     POINT EMPTY
4135     POINT EMPTY
5579     POINT EMPTY
6922     POINT EMPTY
7246     POINT EMPTY
7637     POINT EMPTY
9617     POINT EMPTY
10154    POINT EMPTY
12767    POINT EMPTY
13730    POINT EMPTY
21049    POINT EMPTY
31573    POINT EMPTY
Name: geometry, dtype: geometry

In [ ]:
buffer_gdf[buffer_gdf['Latitude'].isnull()]

,Unnamed: 0.1,Unnamed: 0,Property Name,Property ID,State/Province,Country Name,SNL Global Region,Latitude,Longitude,Population density in a 10km radius -Mean,...,Arid climate,Tropical rainforest climate,Polar climate,Croplands in 10km radius - Mean,Croplands in 20km radius - Mean,Croplands in 100km radius - Mean,Pasture land in a 10 km radius - Mean,Pasture land in a 20 km radius - Mean,Pasture land in a 100 km radius - Mean,geometry
1292,1292,1292,Armiansk,70037,NaN,Ukraine,Europe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
3827,3827,3827,Bravo,68628,West Virginia,USA,United States and Canada,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
4018,4018,4018,BSMDCL,69532,NaN,India,Asia-Pacific,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
4135,4135,4135,Buhler,68766,Pennsylvania,USA,United States and Canada,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
5579,5579,5579,Charolais No 5,68730,Kentucky,USA,United States and Canada,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
6922,6922,6922,Crooks Gap,54952,Wyoming,USA,United States and Canada,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
7246,7246,7246,Dali,56051,Yunnan,China,Asia-Pacific,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
7637,7637,7637,Denise,55405,NaN,Chile,Latin America and Caribbean,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
9617,9617,9617,Farafangana,54286,NaN,Madagascar,Africa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY
10154,10154,10154,Frasure Creek Deep No 15,70125,West Virginia,USA,United States and Canada,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLYGON EMPTY


In [ ]:
buffer_gdf2 = buffer_gdf.to_crs('EPSG:4326')

In [ ]:
gdf_cropland2 = gdf_cropland.reset_index()
gdf_cropland2

,pasture,geometry
0,0.001,"MULTIPOLYGON (((-115.17062 29.74270, -115.1706..."
1,0.002,"MULTIPOLYGON (((-115.58726 29.99268, -115.5872..."
2,0.003,"MULTIPOLYGON (((-115.00396 29.57604, -114.9206..."
3,0.004,"MULTIPOLYGON (((-114.42067 28.99274, -114.3373..."
4,0.005,"MULTIPOLYGON (((-113.83737 27.82615, -113.7540..."
...,...,...
995,0.996,"MULTIPOLYGON (((-104.33795 27.40950, -104.4212..."
996,0.997,"MULTIPOLYGON (((-109.17099 27.90947, -109.2543..."
997,0.998,"MULTIPOLYGON (((-104.50461 27.90947, -104.5879..."
998,0.999,"MULTIPOLYGON (((-105.08790 28.49277, -105.1712..."


In [ ]:
gdf_cropland2['geometry_20'] = gdf_cropland2['geometry']
gdf_cropland2['geometry_100'] = gdf_cropland2['geometry']

In [ ]:
result = gpd.overlay(gdf_cropland2, buffer_gdf2, how='intersection')

In [ ]:
result2 = result.to_crs('EPSG:3857')
result2

,pasture,Unnamed: 0.1,Unnamed: 0,Property Name,Property ID,State/Province,Country Name,SNL Global Region,Latitude,Longitude,...,Arid climate,Tropical rainforest climate,Polar climate,Croplands in 10km radius - Mean,Croplands in 20km radius - Mean,Croplands in 100km radius - Mean,Pasture land in a 10 km radius - Mean,Pasture land in a 20 km radius - Mean,Pasture land in a 100 km radius - Mean,geometry
0,0.001,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6494463.139 -1346249.904, -65..."
1,0.002,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6494463.139 -1365222.716, -65..."
2,0.003,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"POLYGON ((-6568671.606 -1374713.576, -6577947...."
3,0.004,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6466634.964 -1242104.465, -64..."
4,0.005,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6550119.489 -1374713.576, -65..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4686427,1.000,4868,4868,Captain,41820,British Columbia,Canada,United States and Canada,54.83143,-123.93455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-13813273.184 7231859.364, -13813273..."
4686428,1.000,15322,15322,KM,33719,Bayanhongor,Mongolia,Asia-Pacific,43.98675,98.36591,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((10963078.692 5562430.678, 1096..."
4686429,1.000,21069,21069,Musgrave,55204,South Australia,Australia,Asia-Pacific,-27.00690,131.15100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((14692054.152 -3103145.038, 146..."
4686430,1.000,21073,21073,Musgrave Province,55001,South Australia,Australia,Asia-Pacific,-26.82921,131.75996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((14719882.327 -3040907.016, 147..."


In [ ]:
result2['area'] = result2['geometry'].area / 1e6
result2['area_20'] = result2['geometry_20'].area / 1e6
result2['area_100'] = result2['geometry_100'].area / 1e6
result2

,pasture,Unnamed: 0.1,Unnamed: 0,Property Name,Property ID,State/Province,Country Name,SNL Global Region,Latitude,Longitude,...,Tropical rainforest climate,Polar climate,Croplands in 10km radius - Mean,Croplands in 20km radius - Mean,Croplands in 100km radius - Mean,Pasture land in a 10 km radius - Mean,Pasture land in a 20 km radius - Mean,Pasture land in a 100 km radius - Mean,geometry,area
0,0.001,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6494463.139 -1346249.904, -65...",89.246177
1,0.002,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6494463.139 -1365222.716, -65...",175.965704
2,0.003,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"POLYGON ((-6568671.606 -1374713.576, -6577947....",88.037770
3,0.004,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6466634.964 -1242104.465, -64...",324.129512
4,0.005,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6550119.489 -1374713.576, -65...",199.299497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4686427,1.000,4868,4868,Captain,41820,British Columbia,Canada,United States and Canada,54.83143,-123.93455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-13813273.184 7231859.364, -13813273...",3.367837
4686428,1.000,15322,15322,KM,33719,Bayanhongor,Mongolia,Asia-Pacific,43.98675,98.36591,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((10963078.692 5562430.678, 1096...",407.264684
4686429,1.000,21069,21069,Musgrave,55204,South Australia,Australia,Asia-Pacific,-27.00690,131.15100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((14692054.152 -3103145.038, 146...",96.326539
4686430,1.000,21073,21073,Musgrave Province,55001,South Australia,Australia,Asia-Pacific,-26.82921,131.75996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((14719882.327 -3040907.016, 147...",193.508764


In [ ]:
result2['partial_cropland1'] = result2['area'] * result['crop_index']
result2['partial_cropland2'] = result2['area_20'] * result['crop_index']
result2['partial_cropland3'] = result2['area_100'] * result['crop_index']
result2

,pasture,Unnamed: 0.1,Unnamed: 0,Property Name,Property ID,State/Province,Country Name,SNL Global Region,Latitude,Longitude,...,Polar climate,Croplands in 10km radius - Mean,Croplands in 20km radius - Mean,Croplands in 100km radius - Mean,Pasture land in a 10 km radius - Mean,Pasture land in a 20 km radius - Mean,Pasture land in a 100 km radius - Mean,geometry,area,partial_pasture
0,0.001,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6494463.139 -1346249.904, -65...",89.246177,0.089246
1,0.002,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6494463.139 -1365222.716, -65...",175.965704,0.351931
2,0.003,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"POLYGON ((-6568671.606 -1374713.576, -6577947....",88.037770,0.264113
3,0.004,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6466634.964 -1242104.465, -64...",324.129512,1.296518
4,0.005,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.48279,-58.83353,...,NaN,NaN,NaN,NaN,0.44395,0.375373,NaN,"MULTIPOLYGON (((-6550119.489 -1374713.576, -65...",199.299497,0.996497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4686427,1.000,4868,4868,Captain,41820,British Columbia,Canada,United States and Canada,54.83143,-123.93455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-13813273.184 7231859.364, -13813273...",3.367837,3.367837
4686428,1.000,15322,15322,KM,33719,Bayanhongor,Mongolia,Asia-Pacific,43.98675,98.36591,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((10963078.692 5562430.678, 1096...",407.264684,407.264684
4686429,1.000,21069,21069,Musgrave,55204,South Australia,Australia,Asia-Pacific,-27.00690,131.15100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((14692054.152 -3103145.038, 146...",96.326539,96.326539
4686430,1.000,21073,21073,Musgrave Province,55001,South Australia,Australia,Asia-Pacific,-26.82921,131.75996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((14719882.327 -3040907.016, 147...",193.508764,193.508764


In [ ]:
id_cropland_dict1 = dict(result2.groupby('Property ID ')['partial_cropland1'].sum())
id_cropland_dict2 = dict(result2.groupby('Property ID ')['partial_cropland2'].sum())
id_cropland_dict3 = dict(result2.groupby('Property ID ')['partial_cropland3'].sum())

{24463: 17745.10951807511,
 24464: 17445.770743454155,
 24465: 18714.79210190013,
 24466: 18076.03084251411,
 24467: 18455.119253235123,
 24468: 18415.42395126326,
 24469: 16776.029006224366,
 24470: 16275.345615534517,
 24471: 18401.967289242322,
 24472: 17269.675916074775,
 24473: 15681.610930967121,
 24474: 15670.79681926878,
 24475: 19069.49020629155,
 24476: 15543.530909131727,
 24477: 18206.106647060155,
 24478: 17759.50382721702,
 24479: 11062.969742625004,
 24480: 8348.3243866634,
 24481: 17050.34924558267,
 24482: 27767.06137631608,
 24483: 25743.027340530047,
 24484: 26290.94293758223,
 24485: 26052.39563115156,
 24486: 25974.669088802562,
 24487: 26506.393822820493,
 24488: 26111.162108738634,
 24489: 26217.69609738245,
 24490: 21808.58384077948,
 24491: 27827.517807832326,
 24492: 27738.93902314065,
 24493: 26615.412033351546,
 24494: 27764.08714974786,
 24495: 27709.168033212267,
 24496: 27617.01987570045,
 24497: 25968.391178246973,
 24498: 25051.43033881516,
 24499: 7485

In [ ]:
buffer_gdf['area'] = buffer_gdf['geometry'].area / 1e6
buffer_gdf['area_20'] = buffer_gdf['geometry_20'].area / 1e6
buffer_gdf['area_100'] = buffer_gdf['geometry_100'].area / 1e6

In [ ]:
target1 = []
for pi in buffer_gdf['Property ID ']:
    if pi in id_cropland_dict1.keys():
        target1.append(id_cropland_dict1[pi])
    else:
        target1.append(np.NaN)

target1

[4835.8567033710715,
 6428.4313315897425,
 11783.920832739464,
 3929.8460707182303,
 25177.984026802893,
 nan,
 16487.91264254465,
 16487.228967904415,
 nan,
 11386.118741365031,
 nan,
 1792.4616333252125,
 nan,
 6280.668016550464,
 3923.5475530015337,
 9593.360750651522,
 14874.747777345252,
 9597.300446044761,
 160.69614688756297,
 7310.1558912055,
 6143.612667934981,
 3675.4241886271475,
 10327.570627200199,
 nan,
 13495.015941905052,
 nan,
 21982.034840112334,
 nan,
 nan,
 2789.1029907294396,
 902.5089308011619,
 nan,
 27937.197935301057,
 3803.02225331618,
 333.455295638908,
 nan,
 nan,
 8792.500185382243,
 nan,
 113.55293219416654,
 17741.19253077974,
 7324.05172049079,
 8120.745139863477,
 27.343191267221403,
 6535.315845659039,
 5959.517894178202,
 nan,
 15516.398296553301,
 12727.277886578295,
 4162.915105868418,
 3853.817124679102,
 18078.23715288571,
 7443.669145221297,
 566.2810599110325,
 1747.5257338167914,
 15935.498653357317,
 207.83036562048738,
 92.04848362048902,
 35

In [ ]:
target2 = []
for pi in buffer_gdf['Property ID ']:
    if pi in id_cropland_dict2.keys():
        target2.append(id_cropland_dict2[pi])
    else:
        target2.append(np.NaN)

target2

In [ ]:
target3 = []
for pi in buffer_gdf['Property ID ']:
    if pi in id_cropland_dict3.keys():
        target3.append(id_cropland_dict3[pi])
    else:
        target3.append(np.NaN)

target3

In [ ]:
buffer_gdf['area_sum_10km'] = target1
buffer_gdf['area_sum_20km'] = target2
buffer_gdf['area_sum_100km'] = target3

,Unnamed: 0.1,Unnamed: 0,Property Name,Property ID,State/Province,Country Name,SNL Global Region,Latitude,Longitude,Population density in a 10km radius -Mean,...,Polar climate,Croplands in 10km radius - Mean,Croplands in 20km radius - Mean,Croplands in 100km radius - Mean,Pasture land in a 10 km radius - Mean,Pasture land in a 20 km radius - Mean,Pasture land in a 100 km radius - Mean,geometry,area,area_sum_100km
0,0,0,101,64665,Santa Catarina,Brazil,Latin America and Caribbean,-28.732400,-49.245520,NaN,...,NaN,NaN,NaN,NaN,0.151388,0.138442,NaN,"POLYGON ((-5381986.210 -3341630.440, -5382467....",31365.484905,4835.856703
1,1,1,11 RAU-Novaya,68289,Luhans'ka Oblast',Ukraine,Europe,48.519540,38.658270,NaN,...,NaN,NaN,NaN,NaN,0.069485,0.115809,NaN,"POLYGON ((4403418.931 6193727.142, 4402937.404...",31365.484905,6428.431332
2,2,2,1165L,63154,Tete,Mozambique,Africa,-16.230000,34.000000,NaN,...,NaN,NaN,NaN,NaN,0.077611,0.247622,NaN,"POLYGON ((3884862.687 -1831373.468, 3884381.16...",31365.484905,11783.920833
3,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.482790,-58.833530,NaN,...,NaN,NaN,NaN,NaN,0.443950,0.375373,NaN,"POLYGON ((-6449318.601 -1286902.183, -6449800....",31365.484905,3929.846071
4,4,4,12 Mile Creek,66178,Queensland,Australia,Asia-Pacific,-23.163000,149.083000,NaN,...,NaN,NaN,NaN,NaN,0.996582,0.955124,NaN,"POLYGON ((16695843.646 -2651742.668, 16695362....",31365.484905,25177.984027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34264,34264,34264,Zymo,38983,British Columbia,Canada,United States and Canada,54.831119,-128.058003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-14155351.686 7329158.500, -14155833...",31365.484905,NaN
34265,34265,34265,Zyrianovsk Complex,28934,Shyghys Qazaqstan oblysy,Kazakhstan,Europe,49.877040,84.262650,NaN,...,NaN,NaN,NaN,NaN,0.521052,0.513982,NaN,"POLYGON ((9480075.291 6425008.504, 9479593.764...",31365.484905,9897.416317
34266,34266,34266,Zyrox Texada,59481,British Columbia,Canada,United States and Canada,49.775650,-124.600290,NaN,...,NaN,NaN,NaN,NaN,0.001141,0.002642,NaN,"POLYGON ((-13770440.835 6407512.648, -13770922...",31365.484905,83.005307
34267,34267,34267,Zyryanovskoe,61205,Shyghys Qazaqstan oblysy,Kazakhstan,Europe,49.740000,84.296000,NaN,...,NaN,NaN,NaN,NaN,0.706592,0.699720,NaN,"POLYGON ((9483787.796 6401369.585, 9483306.269...",31365.484905,12086.963485


In [ ]:
buffer_gdf['Croplands in 10km radius - Mean'] = buffer_gdf['area_sum_10km'] / buffer_gdf['area']
buffer_gdf['Croplands in 20km radius - Mean'] = buffer_gdf['area_sum_20km'] / buffer_gdf['area_20']
buffer_gdf['Croplands in 100km radius - Mean'] = buffer_gdf['area_sum_100km'] / buffer_gdf['area_100']
buffer_gdf

,Unnamed: 0.1,Unnamed: 0,Property Name,Property ID,State/Province,Country Name,SNL Global Region,Latitude,Longitude,Population density in a 10km radius -Mean,...,Polar climate,Croplands in 10km radius - Mean,Croplands in 20km radius - Mean,Croplands in 100km radius - Mean,Pasture land in a 10 km radius - Mean,Pasture land in a 20 km radius - Mean,Pasture land in a 100 km radius - Mean,geometry,area,area_sum_100km
0,0,0,101,64665,Santa Catarina,Brazil,Latin America and Caribbean,-28.732400,-49.245520,NaN,...,NaN,NaN,NaN,NaN,0.151388,0.138442,0.154178,"POLYGON ((-5381986.210 -3341630.440, -5382467....",31365.484905,4835.856703
1,1,1,11 RAU-Novaya,68289,Luhans'ka Oblast',Ukraine,Europe,48.519540,38.658270,NaN,...,NaN,NaN,NaN,NaN,0.069485,0.115809,0.204952,"POLYGON ((4403418.931 6193727.142, 4402937.404...",31365.484905,6428.431332
2,2,2,1165L,63154,Tete,Mozambique,Africa,-16.230000,34.000000,NaN,...,NaN,NaN,NaN,NaN,0.077611,0.247622,0.375697,"POLYGON ((3884862.687 -1831373.468, 3884381.16...",31365.484905,11783.920833
3,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.482790,-58.833530,NaN,...,NaN,NaN,NaN,NaN,0.443950,0.375373,0.125292,"POLYGON ((-6449318.601 -1286902.183, -6449800....",31365.484905,3929.846071
4,4,4,12 Mile Creek,66178,Queensland,Australia,Asia-Pacific,-23.163000,149.083000,NaN,...,NaN,NaN,NaN,NaN,0.996582,0.955124,0.802729,"POLYGON ((16695843.646 -2651742.668, 16695362....",31365.484905,25177.984027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34264,34264,34264,Zymo,38983,British Columbia,Canada,United States and Canada,54.831119,-128.058003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-14155351.686 7329158.500, -14155833...",31365.484905,NaN
34265,34265,34265,Zyrianovsk Complex,28934,Shyghys Qazaqstan oblysy,Kazakhstan,Europe,49.877040,84.262650,NaN,...,NaN,NaN,NaN,NaN,0.521052,0.513982,0.315551,"POLYGON ((9480075.291 6425008.504, 9479593.764...",31365.484905,9897.416317
34266,34266,34266,Zyrox Texada,59481,British Columbia,Canada,United States and Canada,49.775650,-124.600290,NaN,...,NaN,NaN,NaN,NaN,0.001141,0.002642,0.002646,"POLYGON ((-13770440.835 6407512.648, -13770922...",31365.484905,83.005307
34267,34267,34267,Zyryanovskoe,61205,Shyghys Qazaqstan oblysy,Kazakhstan,Europe,49.740000,84.296000,NaN,...,NaN,NaN,NaN,NaN,0.706592,0.699720,0.385359,"POLYGON ((9483787.796 6401369.585, 9483306.269...",31365.484905,12086.963485


In [ ]:
buffer_gdf.drop(columns = ['geometry','geometry_20','geometry_100', 'area','area_20','area_100', 'area_sum_10km','area_sum_20km','area_sum_100km'], inplace=True)
buffer_gdf

,Unnamed: 0.1,Unnamed: 0,Property Name,Property ID,State/Province,Country Name,SNL Global Region,Latitude,Longitude,Population density in a 10km radius -Mean,...,Current climate category,Arid climate,Tropical rainforest climate,Polar climate,Croplands in 10km radius - Mean,Croplands in 20km radius - Mean,Croplands in 100km radius - Mean,Pasture land in a 10 km radius - Mean,Pasture land in a 20 km radius - Mean,Pasture land in a 100 km radius - Mean
0,0,0,101,64665,Santa Catarina,Brazil,Latin America and Caribbean,-28.732400,-49.245520,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.151388,0.138442,0.154178
1,1,1,11 RAU-Novaya,68289,Luhans'ka Oblast',Ukraine,Europe,48.519540,38.658270,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069485,0.115809,0.204952
2,2,2,1165L,63154,Tete,Mozambique,Africa,-16.230000,34.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.077611,0.247622,0.375697
3,3,3,119 Property,32960,Mato Grosso,Brazil,Latin America and Caribbean,-11.482790,-58.833530,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.443950,0.375373,0.125292
4,4,4,12 Mile Creek,66178,Queensland,Australia,Asia-Pacific,-23.163000,149.083000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.996582,0.955124,0.802729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34264,34264,34264,Zymo,38983,British Columbia,Canada,United States and Canada,54.831119,-128.058003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34265,34265,34265,Zyrianovsk Complex,28934,Shyghys Qazaqstan oblysy,Kazakhstan,Europe,49.877040,84.262650,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.521052,0.513982,0.315551
34266,34266,34266,Zyrox Texada,59481,British Columbia,Canada,United States and Canada,49.775650,-124.600290,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001141,0.002642,0.002646
34267,34267,34267,Zyryanovskoe,61205,Shyghys Qazaqstan oblysy,Kazakhstan,Europe,49.740000,84.296000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.706592,0.699720,0.385359


In [ ]:
buffer_gdf.to_excel('GIS_data_code_c.xlsx')